### My attemp - minimal reproduction - all in one script 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
access_df = pd.read_csv('./data/train.csv')
temp_dir = 'D:/Projects/FailSafe_500/11. AmazonAccess/data/'

In [2]:
from __future__ import division

import argparse
import logging

from sklearn import metrics, linear_model, ensemble
from sklearn.model_selection import train_test_split
import ml
import diagnostics
from data import load_data, save_results
from feature_extraction import create_datasets


In [3]:
y = pd.read_csv(temp_dir+'train.csv')['ACTION']
X = pd.read_csv(temp_dir+'train.csv').drop(['ACTION'],axis=1)
X_test = pd.read_csv(temp_dir+'test.csv').drop(['id'],axis=1)

#### New technique learned to initialize a whole bunch of models and corresponding datasets at once

In [4]:
selected_models = [
                    "LR:tuples_sf",
                    "LR:greedy_sfl",
                    "LR:greedy2_sfl",
                    "LR:greedy3_sf",
                    "RFC:basic_b",
                    "RFC:tuples_f",
                    "RFC:tuples_fd",
                    "RFC:greedy_f",
                    "RFC:greedy2_f",
                    "GBC:basic_f",
                    "GBC:tuples_f",
                    "LR:greedy_sbl",
                    "GBC:greedy_c",
                    "GBC:tuples_cf",
                ]

models = []

for item in selected_models:
    model_id, dataset = item.split(':')
    # instantiating models (with datasets)
    model = {'LR': linear_model.LogisticRegression,
             'GBC': ensemble.GradientBoostingClassifier,
             'RFC': ensemble.RandomForestClassifier,
             'ETC': ensemble.ExtraTreesClassifier}[model_id]() #I have never seen this done before.
    model.set_params(random_state=42)
    models.append((model, dataset))

datasets = [dataset for model, dataset in models]


In [8]:
pd.DataFrame(datasets).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,tuples_sf,greedy_sfl,greedy2_sfl,greedy3_sf,basic_b,tuples_f,tuples_fd,greedy_f,greedy2_f,basic_f,tuples_f,greedy_sbl,greedy_c,tuples_cf


#### Now the various datasets are created. 

There is a scheme to write each of the separate datasets into pickles etc etc but here I will try to take the bare-bones and implement in one script

In [16]:
# feature generation

COLNAMES = ["resource", "manager", "role1", "role2", "department",
            "title", "family_desc", "family"]

dictionaries = []

for i in range(len(COLNAMES)):
    dictionaries.append({'total': 0})

for i, row_vals_list in enumerate(X.values): 
    
    for col in range(len(COLNAMES)):
        dictionaries[col]['total'] += 1
        if row_vals_list[col] not in dictionaries[col]:
            dictionaries[col][row_vals_list[col]] = {'total': 1}
            for k, key in enumerate(COLNAMES):                        
                    dictionaries[col][row_vals_list[col]][key] = {row_vals_list[k]: 1}
        else:
            dictionaries[col][row_vals_list[col]]['total'] += 1
            for k, key in enumerate(COLNAMES):
                if row_vals_list[k] not in dictionaries[col][row_vals_list[col]][key]:
                    dictionaries[col][row_vals_list[col]][key][row_vals_list[k]] = 1
                else:
                    dictionaries[col][row_vals_list[col]][key][row_vals_list[k]] += 1


In [45]:
j = 1
row = X.iloc[0]
print(row[j]) # row 0 col 1
x = dictionaries[j][row[j]] # dictionary entry for row 0 col 1 -- this is all the value_counts of other vals where val == row0/col1
x[COLNAMES[0]]#.get(row[0], 0) if j > 0 and j < 7 else 0
x[COLNAMES[0]].get(row[0], 0) if j > 0 and j < 7 else 0

85475


3

In [23]:
len(dictionaries[1].keys())

4244

In [33]:
len(X['MGR_ID'].unique())

4243

In [34]:
X['MGR_ID'].value_counts().head()

770     152
2270     99
2594     82
1350     71
2014     67
Name: MGR_ID, dtype: int64

In [37]:
X[X['MGR_ID']==770]['RESOURCE'].value_counts()

732       5
19802     4
915       4
34583     4
40729     4
         ..
43906     1
837       1
23867     1
37793     1
103867    1
Name: RESOURCE, Length: 97, dtype: int64

In [35]:
dictionaries[1][770]

{'total': 152,
 'resource': {3069: 3,
  15007: 1,
  915: 4,
  3066: 1,
  24885: 1,
  14912: 1,
  20732: 3,
  34583: 4,
  16082: 1,
  34580: 2,
  45519: 1,
  80877: 2,
  19722: 2,
  80555: 1,
  919: 2,
  75519: 2,
  20731: 1,
  80721: 1,
  6977: 2,
  34581: 3,
  42608: 2,
  741: 1,
  732: 5,
  4673: 1,
  23867: 1,
  34236: 1,
  42015: 2,
  34700: 3,
  18231: 1,
  40058: 1,
  29303: 1,
  44812: 2,
  917: 1,
  52688: 3,
  42610: 1,
  80483: 1,
  692: 1,
  15728: 1,
  16070: 3,
  25993: 1,
  31200: 1,
  4675: 3,
  33896: 1,
  37260: 1,
  111102: 1,
  35708: 3,
  40729: 4,
  941: 3,
  42304: 2,
  45904: 1,
  34716: 2,
  19802: 4,
  42611: 1,
  18254: 1,
  80695: 1,
  80487: 2,
  79092: 1,
  34584: 3,
  43906: 1,
  74646: 2,
  837: 1,
  46022: 1,
  37793: 1,
  16431: 1,
  79694: 1,
  22328: 1,
  80906: 1,
  31044: 3,
  3716: 2,
  23164: 1,
  74101: 1,
  83217: 1,
  740: 1,
  71439: 1,
  20312: 1,
  79363: 3,
  42612: 1,
  81250: 1,
  40062: 1,
  976: 1,
  18229: 1,
  673: 2,
  42199: 1,
  22